In [1]:
include("SpinGlassCore_3d.jl")
using .SpinGlassCore
include("../../logger/logger.jl")
using .MyLogger
include("../utils.jl")
using .Utils
using Logging, LinearAlgebra, Statistics, StatsBase, Random
using CairoMakie

seed = 42
Random.seed!(seed)

init_logger(dir="../../logs", level=:Info)
println(names(SpinGlassCore))

[:SpinGlassCore, :calc_acceptance_and_rtt, :calc_energy, :initial_state, :metropolis_sampler, :score_method]


In [2]:
N = 8
β_min = 0.1
β_max = 1.8
M = 20
mcSteps=10^7
eqSteps=10^2
Jh = randn(N, N, N)
Jv = randn(N, N, N)
Jz = randn(N, N, N)
@info """
Hyperparameters of Gaussian model:
--------------------
system size: $(N*N*N),
the number of replica: $M,
MCS: $mcSteps
"""
println("Set hyperparameters.")

Set hyperparameters.


In [3]:
β_ladder_trace = [[0.04637974303856932, 0.08911569265681538, 0.131881867355154, 0.1751897697294175, 0.2192223514289654, 0.26482473397183715, 0.3113286548938571, 0.3593087735175781, 0.4093502963093775, 0.46243510053762077, 0.5180829471816236, 0.5782182390931703, 0.6471490210116588, 0.725708477037973, 0.8158184261336104, 0.9230535353743115, 1.056720737740695, 1.2390291680333412, 1.4984369140477953, 1.880176437241186]]
geometric_β_ladder = Utils.set_temperature_ladder(
    β_min, 
    β_max; 
    M=M, 
    method=:geometric
)
inverse_linear_β_ladder = Utils.set_temperature_ladder(
    β_min, 
    β_max; 
    M=M, 
    method=:inverse_linear
)
push!(β_ladder_trace, geometric_β_ladder)
push!(β_ladder_trace, inverse_linear_β_ladder)

3-element Vector{Vector{Float64}}:
 [0.04637974303856932, 0.08911569265681538, 0.131881867355154, 0.1751897697294175, 0.2192223514289654, 0.26482473397183715, 0.3113286548938571, 0.3593087735175781, 0.4093502963093775, 0.46243510053762077, 0.5180829471816236, 0.5782182390931703, 0.6471490210116588, 0.725708477037973, 0.8158184261336104, 0.9230535353743115, 1.056720737740695, 1.2390291680333412, 1.4984369140477953, 1.880176437241186]
 [0.1000000000000001, 0.11643055668833711, 0.13556074530756068, 0.1578341304124516, 0.18376715668341317, 0.21396112353682642, 0.24911612723054746, 0.29004729373495225, 0.3377036787550609, 0.39319027313151067, 0.45779362385141065, 0.5330116647339088, 0.6205884484634625, 0.7225545852895225, 0.841274326029696, 0.9795003810724302, 1.1404377464470126, 1.3278180168721818, 1.545985908852318, 1.7999999999999998]
 [0.1, 0.18947368421052632, 0.2789473684210526, 0.368421052631579, 0.45789473684210524, 0.5473684210526316, 0.6368421052631579, 0.7263157894736842, 0.81578

In [4]:
mean_rtt_trace = []
strategys = ["uniform acceptance", "geometric", "inverse linear"]

for (β_ladder, strategy) in zip(β_ladder_trace, strategys)
    exchange_prob_means, all_rtts = SpinGlassCore.calc_acceptance_and_rtt(
        N, 
        β_ladder,
        Jh,
        Jv,
        Jz;
        mcSteps=mcSteps,
        eqSteps=eqSteps
    )
    mean_rtt = mean(all_rtts)

    @info """
        $strategy: 
        inverse temperature ladder = $β_ladder, 
        Mean Round-Trip Time: $mean_rtt ± $(Statistics.std(all_rtts))"""
    println("""
        $strategy: 
        inverse temperature ladder = $β_ladder, 
        Mean Round-Trip Time: $mean_rtt ± $(Statistics.std(all_rtts))"""
    )
end

uniform acceptance: 
inverse temperature ladder = [0.04637974303856932, 0.08911569265681538, 0.131881867355154, 0.1751897697294175, 0.2192223514289654, 0.26482473397183715, 0.3113286548938571, 0.3593087735175781, 0.4093502963093775, 0.46243510053762077, 0.5180829471816236, 0.5782182390931703, 0.6471490210116588, 0.725708477037973, 0.8158184261336104, 0.9230535353743115, 1.056720737740695, 1.2390291680333412, 1.4984369140477953, 1.880176437241186], 
Mean Round-Trip Time: 20879.82489539749 ± 15342.586798200198
geometric: 
inverse temperature ladder = [0.1000000000000001, 0.11643055668833711, 0.13556074530756068, 0.1578341304124516, 0.18376715668341317, 0.21396112353682642, 0.24911612723054746, 0.29004729373495225, 0.3377036787550609, 0.39319027313151067, 0.45779362385141065, 0.5330116647339088, 0.6205884484634625, 0.7225545852895225, 0.841274326029696, 0.9795003810724302, 1.1404377464470126, 1.3278180168721818, 1.545985908852318, 1.7999999999999998], 
Mean Round-Trip Time: 25835.75981344